This script is used to sort the images such that within one cycle folder all wells have the same antibodies.  
Files are first moved to a "temp" folder (`dir_temp`) and from there moved back into the original folder but into the right order, specified by `AB_set_order_permuted`. The temp folder can be removed manually once resorting is complete.  
To check the AB orders of individual wells, `dir_AB` specifies the path to a file containing this information. This .txt file contains three columns ("ID", "AB-order", "well") but technically the "ID" well is not required for this notebook to work.

In [6]:
from pathlib import Path
import pandas as pd

In [18]:
## Define parameters for this experiment

experiment_name = "plate14"

# Create dictionary for cycle name conversion (plate14: cycle21 last in both orders so we can ignore it here)
cycles_before = ["cycle{}".format(n) for n in range(1,21)]  # Folder the images are now in (cycle1 - cycle20)
AB_set_order_permuted = [6,5,3,7,2,4,1,14,13,9,10,12,11,8,18,15,20,17,19,16]  # Actual order, i.e. cycle1 should actually be cycle6, cycle2 should be cycle5 etc.
cycles_after = ["cycle{}".format(n) for n in AB_set_order_permuted]
cycle_dict = dict(zip(cycles_before, cycles_after))  # Images will be moved from key cycle to value cycle

nchan = 4  # number of channels/images per well

# Path to input folder
dir_in = Path(r"/links/groups/treutlein/DATA/imaging/PW/4i/plate14_aligned_UZH")
# Path to temp folder
dir_temp = Path(str(dir_in)+"_temp")
# Path to file containing AB orders of wells
dir_AB = Path(r"/links/groups/treutlein/DATA/imaging/PW/4i/plate14/accessory_data/ID_ABorder_well.txt")

In [3]:
# Load AB order table
AB_df = pd.read_csv(dir_AB)
# Remove rows with missing values/wells
AB_df.dropna(inplace=True)
# Only keep the wells with AB order 2 (permuted order)
AB_df = AB_df[AB_df["AB-order"] == 2]
# Create a list from remaining well names
well_names = AB_df["well"]

In [4]:
# Move all relevant files to a temp folder
print("Moving files to temp folder...")
for cycle_old, cycle_new in cycle_dict.items():
    print("{} -> {}".format(cycle_old, cycle_new))
    # Create folder for cycle in output directory
    Path(dir_temp/cycle_new).mkdir(parents=True, exist_ok=True)
    for well in well_names:
        #print(well)
        for chan in range(1, nchan+1):
            filename = "{}_{}_C{:02d}.png".format(experiment_name, well, chan)
            file_old = dir_in/cycle_old/filename
            file_new = dir_temp/cycle_new/filename
            # If output file already exists, skip ahead
            if file_new.is_file():
                print("File at destination already exists")
                continue
            try:
                # Move file to new location
                file_old.rename(str(file_new))
            except FileNotFoundError:
                print("{} well {} not found".format(cycle_old, well))

cycle1 -> cycle6
cycle2 -> cycle5
cycle3 -> cycle3
cycle4 -> cycle7
cycle5 -> cycle2
cycle6 -> cycle4
cycle7 -> cycle1
cycle8 -> cycle14
cycle9 -> cycle13
cycle10 -> cycle9
cycle11 -> cycle10
cycle12 -> cycle12
cycle13 -> cycle11
cycle14 -> cycle8
cycle15 -> cycle18
cycle16 -> cycle15
cycle17 -> cycle20
cycle18 -> cycle17
cycle19 -> cycle19
cycle20 -> cycle16


In [7]:
# Move files from temp folder back to input folder but into the right cycle
print("Moving files from temp folder to right cycle in original folder")
for cycle in cycle_dict:
    print(cycle)
    # Get all files in folder
    files = Path(dir_temp/cycle).glob("*.png")
    for file in files:
        name = file.name  # filename
        # Get paths to file in temp folder and to its destination
        file_old = dir_temp/cycle/name
        file_new = dir_in/cycle/name
        # If output file already exists, skip ahead
        if file_new.is_file():
            Print("File at destination already exists")
            continue
        try:
            # Move file to new location
            file_old.rename(str(file_new))
        except FileNotFoundError:
            print("{} well {} not found".format(cycle, well))

Moving files from temp folder to right cycle in original folder
